In [1]:
import os
import random
from utils.hand_model_lite import HandModelMJCFLite
import numpy as np
import transforms3d
import torch
import trimesh


In [2]:
mesh_path = "../data/meshdata"
data_path = "../data/dataset"

use_visual_mesh = False

hand_file = "mjcf/DP-Flex_opened_kinematics.xml"

joint_names = [
    'Joint_left_abduction', 'Joint_left_flexion', 'Joint_left_finray_proxy',
        'Joint_left_dynamixel_crank', 'Joint_left_crank_pusher',
        'Joint_right_abduction', 'Joint_right_flexion', 'Joint_right_finray_proxy',
        'Joint_right_dynamixel_crank', 'Joint_right_crank_pusher',
        'Joint_thumb_rotation', 'Joint_thumb_abduction', 'Joint_thumb_flexion', 'Joint_thumb_finray_proxy',
        'Joint_thumb_dynamixel_crank', 'Joint_thumb_crank_pusher'
]
translation_names = ['WRJTx', 'WRJTy', 'WRJTz']
rot_names = ['WRJRx', 'WRJRy', 'WRJRz']


In [3]:
hand_model = HandModelMJCFLite(
    hand_file,
    "mjcf/assets")


In [6]:
grasp_code_list = []
for code in os.listdir(data_path):
    grasp_code_list.append(code[:-4])


In [7]:
grasp_code = random.choice(grasp_code_list)
grasp_data = np.load(
    os.path.join(data_path, grasp_code+".npy"), allow_pickle=True)
object_mesh_origin = trimesh.load(os.path.join(
    mesh_path, grasp_code, "coacd/decomposed.obj"))
print(grasp_code)


core-bottle-1a7ba1f4c892e2da30711cdbdbc73924


In [8]:
index = random.randint(0, len(grasp_data) - 1)


qpos = grasp_data[index]['qpos']
rot = np.array(transforms3d.euler.euler2mat(
    *[qpos[name] for name in rot_names]))
rot = rot[:, :2].T.ravel().tolist()
hand_pose = torch.tensor([qpos[name] for name in translation_names] + rot + [qpos[name]
                         for name in joint_names], dtype=torch.float, device="cpu").unsqueeze(0)
hand_model.set_parameters(hand_pose)
hand_mesh = hand_model.get_trimesh_data(0)
object_mesh = object_mesh_origin.copy().apply_scale(grasp_data[index]["scale"])


In [9]:
(hand_mesh+object_mesh).show()
# (hand_mesh).show()
# (object_mesh).show()